## Import libraries

In [1]:
import cv2
import albumentations as A

In [2]:
import random
random.seed(7)

In [3]:
category_ids = [0, 1, 2]
category_id_to_name = {'blue': 0, 'purple': 1, 'red': 2}

## Set image and label directory

In [4]:
import os
IMAGE_DIR = 'images'
LABEL_DIR = 'labels'
img_files = os.listdir(IMAGE_DIR)
label_files = os.listdir(LABEL_DIR)

In [5]:
transform = A.Compose(
  [
    A.HorizontalFlip(p=0.8),
    A.VerticalFlip(p=0.2),
    A.OneOf([
      A.Blur(blur_limit=5, p=0.5),
      A.MotionBlur(blur_limit=5, p=0.8)
    ], p=1.0)
  ], bbox_params=A.BboxParams(format='yolo',  min_visibility=0.3, min_area=3600., label_fields=[])
)  

In [14]:
for i, img_file in enumerate(img_files):

  # Split the filename and extension
  file_name, file_extension = os.path.splitext(img_file)

  img_path = os.path.join(IMAGE_DIR, f'{file_name}.jpg')
  label_path = os.path.join(LABEL_DIR, f'{file_name}.txt')

  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
  
  bboxes = []
  saved_category_ids = []

  # Read from label text file
  with open(label_path, 'r') as file:
    lines = file.readlines()

  for line in lines:

    parts = line.strip().split()
    class_num = int(parts[0])
    bbox_coordinates = [float(part) for part in parts[1:]]

    saved_category_ids.append(class_num)
    bboxes.append(bbox_coordinates) 
  
  images_list = []
  saved_bboxes = []
  
  out_img_paths = []
  out_label_paths = []

  # for thrice the dataset: 2 augmentations for a single image
  for i in range(2):
    
    outfile = f'{file_name}-aug{i}'
    out_img_path = os.path.join(IMAGE_DIR, f'{outfile}.jpg')
    out_label_path = os.path.join(LABEL_DIR, f'{outfile}.txt')

    transformed = transform(image=img, bboxes=bboxes)

    if len(transformed["bboxes"]) == 0:
      continue

    images_list.append(transformed["image"])
    saved_bboxes.append(transformed["bboxes"])

    out_img_paths.append(out_img_path)
    out_label_paths.append(out_label_path)
    

  breakpoint()
  # Write the augmented image to images directory
  for img in images_list:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_img_path, img)
  
  print(saved_bboxes)

  # Write the labels to label text file and put into labels directory
  for aug_bboxes, out_label_path in zip(saved_bboxes, out_label_paths):
    with open(out_label_path, 'a') as txt_file:
      
      for category_id, aug_bbox in zip(saved_category_ids, aug_bboxes):
        labels = []
        labels.append(str(category_id))
        bbox_info = [str(info) for info in aug_bbox]
        labels += bbox_info
      
      label_txt = ' '.join(labels)

      txt_file.write(label_txt + '\n')

[[(0.8067602040816326, 0.7252551020408161, 0.38647959183673486, 0.5341836734693877), (0.8040816326530611, 0.4926020408163266, 0.3918367346938777, 0.5198979591836735)], [(0.19323979591836743, 0.2747448979591839, 0.38647959183673486, 0.5341836734693877), (0.19591836734693885, 0.5073979591836735, 0.3918367346938777, 0.5198979591836735)]]
[[(0.2565051020408162, 0.31224489795918375, 0.13698979591836746, 0.17346938775510212)], [(0.2565051020408162, 0.31224489795918375, 0.13698979591836746, 0.17346938775510212)]]
